In [1]:
import pymongo
import os
from dotenv import load_dotenv
load_dotenv(override=True) 

True

In [2]:
db_client = pymongo.MongoClient(os.environ['REMOTE_MONGO_DB'])
db = db_client['db_certificates']   
tb_cert_metadata = db['tb_cert_metadata']    #meta data of certificates

In [3]:
#get linkedin page details
tb_linkedin_page = db['tb_linkedin_page']
pages_info = tb_linkedin_page.find({})
page_info_map = {}  #key: category, value: dict of LI page info {link,name}
for info in pages_info:
    page_info_map[info['category']] = {
        'li_page_name': info['name'],
        'li_page_url': info['link']
    }
# print(page_info_map)

https://certification-pro.blogspot.com/

In [4]:
def generate_post_for_cert(_symbol):
    cert_info = tb_cert_metadata.find_one({'symbol': _symbol})
    if cert_info is None:
        return
    if 'short_brief' in cert_info:
        print('<div style="text-align: justify;">' + cert_info['short_brief'] + '</div><br/><br/>')
    #search and print random 10 questions
    tb_questions = db[cert_info['collection_name']]
    pipeline = [
                {"$match": {'type': 'multiple-choice'}},
                {"$sample": {"size": 10}}
            ]
    random_documents = list(tb_questions.aggregate(pipeline))
    index = 1
    for cert in random_documents:
        #title
        print('<div><b>---------- Question '+ str(index) + '</b></div>')
        print('<div style="text-align: justify;">'+ cert['question'] + '</div>')
        print('<ol>')
        for k,v in cert['options'].items():
            print('<li>'+ v +'</li>')
        print('</ol><br/>')
        index = index + 1

    #print bottom lines
    print('<hr/><div>Are they useful?</div>')
    total_question = 6 * cert_info['multi_choice_questions']    #6 exams
    print('<div>Click <a href="'+cert_info['udemy_link']+'" style="background-color: #fcff01;" target="_blank">here</a> to get <span style="background-color: #cfe2f3;"><b>'+str(total_question)+'</b> <b>more questions</b></span> to pass this certification at the first try! Explanation for each answer is included!</div><br/>')
    #follow
    if 'category' in cert_info:
        print('<div>Follow the below LINKEDIN channel to stay updated about 89+ exams!</div>')
        print('<div><a href="'+page_info_map[cert_info['category']]['li_page_url']+'" style="background-color: #d9ead3;" target="_blank">'+page_info_map[cert_info['category']]['li_page_name']+'</a></div><br/>')
    #related certs
    # print('<br/><h3><b><span style="background-color: #3d85c6; color: white; padding: 10px;">RELATED CERTIFICATIONS</span></b></h3><br/>')
    #table
    # print('<table><tr><td>xx</td><td>xx</td><td>xx</td></tr></table>')
#test


In [ ]:
# generate_post_for_cert('SCRUM_PSK_1')

In [8]:
def generate_question_for_mobile_app(_symbol):
    cert_info = tb_cert_metadata.find_one({'symbol': _symbol})
    if cert_info is None:
        return
    #search and print random 10 questions
    tb_questions = db[cert_info['collection_name']]
    pipeline = [
                {"$match": {'type': 'multiple-choice'}},
                {"$sample": {"size": 40}}   #hard code because there are some exams have 200 questions
            ]
    random_documents = list(tb_questions.aggregate(pipeline))
    index = 1
    new_list = []
    for cert in random_documents:
        new_list.append({
            'q': cert['question'],
            'o': cert['options'],
            'a': cert['answer'],
            'e': cert['explanation']
        })
    print(new_list)
#test
# generate_question_for_mobile_app('PMP')

In [15]:
CERTIFICATION_CATEGORIES = {
    "AGILE": 0,
    "AI": 1,
    "CLOUD": 2,
    "DATA_ENGINEER": 3,
    "DIGITAL_INTELLIGENCE": 4,
    "PROJECT_MANAGEMENT": 5,
    "SECURITY": 6,
    "SOLUTIONS_ARCHITECT" : 7
}

In [ ]:
#print list of certification in new format
def print_cert_metadata():
    certs = tb_cert_metadata.find() #find all
    new_list = []
    for cert in certs:
        new_list.append({
            "s": cert['symbol'],
            "n": cert['name'],
            "c": CERTIFICATION_CATEGORIES[cert['category']],
            "l": cert['udemy_link'].replace('https://www.udemy.com/course/', ''),
            'f': ''
        })
    print(new_list)
#test
# print_cert_metadata()


[{'s': 'AWS_SAA', 'n': 'AWS Certified Solutions Architect - Associate (SAA-C03)', 'c': 7, 'l': 'aws-certified-solutions-architect-associate-practice-exams-iq/?referralCode=B5AED1732923B9C511E4', 'f': ''}, {'s': 'AWS_CLF_C02', 'n': 'AWS Certified Cloud Practitioner (CLF-C02)', 'c': 2, 'l': 'aws-cloud-practitioner-practice-exam-tests-2025-clf-c02/?referralCode=1EA374CBBB8A017491C8', 'f': ''}, {'s': 'PMP', 'n': 'PMP Project Management Professional', 'c': 5, 'l': 'pmp-practice-exam-tests-2025-1080-questions/?referralCode=F8F6CF10C2416D94B80E', 'f': ''}, {'s': 'PSM_1', 'n': 'PSM I Professional Scrum Master', 'c': 0, 'l': 'psm-i-scrum-master-practice-exam-tests-2025-480-questions/?referralCode=FC8216569270808C6198', 'f': ''}, {'s': 'GCP_PCDataE', 'n': 'GCP Google Cloud Platform - Data Engineer Professional', 'c': 3, 'l': '6-google-data-engineer-certificate-practice-exam-tests-2025/?referralCode=09680FA83987421C1836', 'f': ''}, {'s': 'AWS_DVA_C02', 'n': 'AWS Certified Developer - Associate (D

In [ ]:
db_client.close()